# IPython interface for running GPU DSM + fitting on cluster

In [38]:
from dsm_tools import ssh_init, job_start, jobs_done, job_transfer_clean, uuid, os, server, username, password
from fdt_fit import fdt_fit, fdt, fdtvec
import numpy as np
import matplotlib.pyplot as plt

** Choose input parameters for GPU DSM **

In [112]:
class Calculation:
    #Input parameters
    beta = 1
    nChains = 5000
    velocityGradient = (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
    CDtoggle = 0
    PDtoggle = 0
    calcMode = 1
    timeStep = 1
    simTime = 100000
    token = None #folder name
    ssh = None #storing Paramiko SSH object
    jobIds = None #array of job IDs on cluster
    jobStatus = 0
    #Calculation options (number of GPU)
    fdt_x = None
    fdt_y = None
    lambdaArr = None
    gArr = None

    #Function to generate input file
    def generate_token(self):
        self.token = str(uuid.uuid4())
        return

    #Function to start calculation on cluster
    def calc(self):
        self.ssh=ssh_init()
        self.jobIds = job_start(self.ssh, self.token)
        self.jobStatus = 1
        return

    #Function to check the status of calculation
    def check_self(self):
        if jobStatus==0:
            if not self.ssh:
                self.ssh=ssh_init()
            if jobs_done(self.ssh, self.jobIds):
                jobStatus==1
                print('Calculation is done. Transfering results')
                job_transfer_clean(self.ssh, self.token)

                #Read f_d(t) simulation rtesults and fit
                with open(self.token + '/fdt_aver.dat') as f:
                    lines = f.readlines()
                    self.fdt_x = np.array([float(line.split()[0]) for line in lines])
                    ydata = np.array([float(line.split()[1]) for line in lines])

                self.fdt_y = 1.0-np.cumsum(ydata)/np.sum(ydata)

                with open(self.token + '/fdt_MMM_fit.dat') as f:
                    lines = f.readlines()
                self.lambdaArr = np.array([float(line.split()[0]) for line in lines[1:]])
                self.gArr = np.array([float(line.split()[1]) for line in lines[1:]])
            else:
                print('Still running, check back later')
        else:
            print('Calculation is finished')

    #Initialize object of class
    def __init__(self):
        self.generate_token()
        if not os.path.exists(self.token):
            os.makedirs(self.token)
        self.generate_input()


class CalculationStar(Calculation):
    #Type of chain
    architecture = 'star'
    nArms = 3
    nkArms = [6, 6, 6]

    def generate_input(self):
        file = open(self.token + "/input.dat","w")

        file.write(str(self.beta)+'\n')
        file.write(self.architecture+'\n')
        file.write(str(self.nArms)+'\n')
        file.write(" ".join(map(str,self.nkArms)) + '\n')
        file.write(str(self.nChains)+'\n')
        file.write(" ".join(map(str,self.velocityGradient)) + '\n')
        file.write(str(self.CDtoggle)+'\n')
        file.write(str(self.PDtoggle)+'\n')
        file.write(str(self.calcMode)+'\n')
        file.write(str(self.timeStep)+'\n')
        file.write(str(self.simTime))

        file.close()

class CalculationLinear(Calculation):
    #Type of chain
    architecture = 'linear'
    nk = 25

    def generate_input(self):
        file = open(self.token + "/input.dat","w")

        file.write(str(self.beta)+'\n')
        file.write(self.architecture+'\n')
        file.write(str(self.nk)+'\n')
        file.write(str(self.nChains)+'\n')
        file.write(" ".join(map(str,self.velocityGradient)) + '\n')
        file.write(str(self.CDtoggle)+'\n')
        file.write(str(self.PDtoggle)+'\n')
        file.write(str(self.calcMode)+'\n')
        file.write(str(self.timeStep)+'\n')
        file.write(str(self.simTime))

        file.close()

In [94]:
x = CalculationLinear()

In [95]:
x.token

In [96]:
x.calc()

In [107]:
x.check_self()

### Check results of simulation and fitting visually

In [137]:
fig = plt.figure(figsize=(8, 6))

ax1 = fig.add_subplot(111)

ax1.set_title("Entanglement lifetime distribution")
ax1.set_xlabel(r'$t/\tau_c$')
ax1.set_ylabel(r'$f_d(t)$')

ax1.scatter(xdata,ydata, c='r', label=r'$f_d(t)$')
ax1.plot(xdata,fdtvec(time=xdata, params=np.append(lambdaArr, gArr)), c='b', label=r'MMM fit')
#plt.ylim(ymin=1e-10)
leg = ax1.legend()
ax1.set_xscale('log')
ax1.set_yscale('log')

plt.show()

In [129]:
lambdaArr

In [130]:
gArr

In [140]:
np.multiply(lambdaArr, gArr)/np.dot(lambdaArr, gArr)

In [142]:
fig1 = plt.figure(figsize=(20, 6))
ax0 = fig1.add_subplot(131)

ax0.set_title(r'$p^{eq}\left(\tau\right)$')
ax0.set_xlabel(r'$\lambda$')
ax0.set_ylabel(r'$g$')

#ax0.scatter(lambdaArr ,gArr, c='b', label=r'Second fit')
ax0.scatter(lambdaArr ,np.multiply(lambdaArr, gArr)/np.dot(lambdaArr, gArr), c='b', label=r'Second fit')

leg = ax0.legend()
ax0.set_xscale('log')
ax0.set_yscale('log')
plt.xlim(xmin=1, xmax=1e4)
plt.ylim(ymin=1e-2, ymax=1)

plt.show()

In [113]:
y = CalculationStar()

In [143]:
y.token

In [115]:
y.calc()

In [119]:
y.check_self()

In [149]:
with open(y.token + '/fdt.dat') as f:
    lines = f.readlines()
    xxdata = np.array([float(line.split()[0]) for line in lines])
    yydata = np.array([float(line.split()[1]) for line in lines])

xdata = 10**(xxdata/1000.0-10.0)
ydata = 1.0-np.cumsum(yydata)/np.sum(yydata)

with open(y.token + '/fdt_MMM_fit.dat') as f:
    lines = f.readlines()
lambdaArr = np.array([float(line.split()[0]) for line in lines[1:]])
gArr = np.array([float(line.split()[1]) for line in lines[1:]])

fig = plt.figure(figsize=(8, 6))

ax1 = fig.add_subplot(111)

ax1.set_title("Entanglement lifetime distribution")
ax1.set_xlabel(r'$t/\tau_c$')
ax1.set_ylabel(r'$f_d(t)$')

ax1.scatter(xdata,ydata, c='r', label=r'$f_d(t)$')
ax1.plot(xdata,fdtvec(time=xdata, params=np.append(lambdaArr, gArr)), c='b', label=r'MMM fit')
#plt.ylim(ymin=1e-10)
leg = ax1.legend()
ax1.set_xscale('log')
ax1.set_yscale('log')

plt.show()

In [90]:
x2=x
x2.CDtoggle = 1

In [91]:
x2.CDtoggle

In [30]:
beta = 1
architecture = 'linear'
#nArms = 3
nkArms = [45]
nChains = 5000
velocityGradient = (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
CDtoggle = 0
PDtoggle = 0
calcMode = 1
timeStep = 1
simTime = 100000